In [2]:
from pyquery import PyQuery as pq
import requests
import re
import uucsv

In [3]:
votelist = []    # to be billno, chamber, membername, vote, points
floorvotevalue = 3

In [4]:
# wanted votes format: List of tuples with billno, chamber, 1 or -1 for multiplier/status, URL to rollall
# Note rollcalls can have late votes and we need to get sponsors, which will go in a supplemental
# that we handcode because we hate our lives

'''   Amended at end, not desired
   ("SB374", "House", 1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17104&BillId=56972&SessionId=83&"
    ),
    ("SB374", "Senate", 1,
    "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17100&BillId=56972&SessionId=83&"
    ),'''

wantedvotes = [
    ("HB111", "House", -1,
    "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17008&BillId=56752&SessionId=83&"),
    ("HB111", "Senate", -1,
    "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17119&BillId=56752&SessionId=83&"),

    ("SB118", "House", -1,
    "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17428&BillId=56694&SessionId=83&"),
    ("SB118", "Senate", -1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17035&BillId=56694&SessionId=83&"),
 
    ("HB351", "House", -1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17251&BillId=57004&SessionId=83&"),
    
    ("HB441", "House", 1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17187&BillId=57076&SessionId=83&"),
    ("HB441", "Senate", 1,
      "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17710&BillId=57076&SessionId=83&"),
    
    ("HB843", "House", -1,
    "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17499&BillId=58235&SessionId=83&"),
    
    ("SB1018", "House", 1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17538&BillId=58321&SessionId=83&"),
    ("SB1018", "Senate", 1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17483&BillId=58321&SessionId=83&"),
    
    ("HB7093", "House", -1,
     "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17206&BillId=59568&SessionId=83&"),
    ("HB7093", "Senate", -1,
    "http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17579&BillId=59568&SessionId=83&")
        
]

In [5]:
for tally in wantedvotes:
    billno, chamber, multiplier, sourceurl = tally
    print("Working on " + billno + " for the " + chamber)
    r = requests.get(sourceurl)
    html = r.content
    yeas = pq(html)("span#ctl00_ContentPlaceHolder1_lblYeas").text().strip()
    nays = pq(html)("span#ctl00_ContentPlaceHolder1_lblNays").text().strip()
    abstained = pq(html)("span#ctl00_ContentPlaceHolder1_lblAbstained").text().strip()
    missed = pq(html)("span#ctl00_ContentPlaceHolder1_lblMissed").text().strip()
    total = pq(html)("span#ctl00_ContentPlaceHolder1_lblTotal").text().strip()
    print(str(int(yeas)+int(nays)+int(abstained)+int(missed)) + " votes found. Y-N-A-M-Total:")    
    print(yeas, nays, abstained, missed, total)
    h = pq(html)("table#ctl00_ContentPlaceHolder1_dlVoteMember")
    for pol in pq(h)("td"):
        if len(pq(pol)("span")) > 0:
            # print(pq(pol))
            # print(pq(pol)("span")[0].text_content())
            vote = pq(pol)("span")[0].text_content().strip()
            member = pq(pol)("span")[1].text_content().strip().encode("utf-8")
            if member == "NuÃ±ez":
                print("HEY! PROBLEM WITH NUNEZ AGAIN")
                member = u"Nuñez"
                member = u"Nunez"
            memberid = pq(pq(pol)("span")[1]).attr("id")
            memberid = memberid.replace("ctl00_ContentPlaceHolder1_dlVoteMember_ctl", "")
            memberid = memberid.replace("_lblVoterName", "")
            points = 0
            if vote == "Y":
                points = floorvotevalue * multiplier
            elif vote == "N":
                points = floorvotevalue * multiplier * -1
            else:
                print("Vote \'" + vote + "\' found for " + member + " on " + billno)
            entry = [billno, chamber, member, vote, str(points)]
            # print(member, vote, memberid)
            votelist.append(entry)
    if "Vote After Roll Call" in str(pq(html)):
        print("After-roll call votes need to be manually added for:\n\t"+ sourceurl)
    
    

Working on HB111 for the House
120 votes found. Y-N-A-M-Total:
('113', '3', '0', '4', '116')
Vote '-' found for Abruzzo on HB111
Vote '-' found for Santiago on HB111
Vote '-' found for Harrell on HB111
HEY! PROBLEM WITH NUNEZ AGAIN
Vote '-' found for Raulerson on HB111
After-roll call votes need to be manually added for:
	http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17008&BillId=56752&SessionId=83&
Working on HB111 for the Senate
40 votes found. Y-N-A-M-Total:
('34', '3', '0', '3', '37')
Vote '-' found for Hukill on HB111
Vote '-' found for Lee on HB111
Vote '-' found for Galvano on HB111
Working on SB118 for the House
120 votes found. Y-N-A-M-Total:
('118', '0', '0', '2', '118')
HEY! PROBLEM WITH NUNEZ AGAIN
Vote '-' found for Stafford on SB118
Vote '-' found for Plasencia on SB118
After-roll call votes need to be manually added for:
	http://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17428&BillId=56694&SessionId=83&
Working on SB118 for the Senat

In [6]:
# OK, so there can be fewer than 120 reps -- someone died, quit? Not marked absent. GOod to know.
# If number of voters is less than or equal to 40, it's Senate; otherwise, House

In [7]:
with open('autovotes.csv', 'wb') as csvfile:
    put = uucsv.UnicodeWriter(csvfile)
    put.writerow(["billno", "chamber", "member", "vote", "points", "potential"])
    for entry in votelist:
        entry.append("2")
        put.writerow(entry)
